In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [3]:
with open('../data/molecules.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [4]:
structures = pd.read_feather('../data/structures.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [21]:
from models import SKModel, partition_data

#data_df = labelled[labelled.type == '1JHC'].head(10)
data_df = labelled.head(4)
train_df, test_df = partition_data(data_df, train_frac=1)
        
model = SKModel(molecules=molecules,
                structures=structures)
model.corr(train_df, train_df)

,id,atom_index_0,atom_index_1,scalar_coupling_constant,coupling_distance,coupling_1JHC,coupling_2JHH,atom0_N,atom0_F,atom0_O,...,atom3_weight,bond01_dist,bond01_valency,bond01_strength,bond12_dist,bond12_valency,bond12_strength,bond23_dist,bond23_valency,bond23_strength
id,1.000000,NaN,0.982708,-0.774582,0.774624,-0.774597,0.774597,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.774595,0.774597,0.774597,NaN,NaN,NaN
atom_index_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_1,0.982708,NaN,1.000000,-0.878299,0.878331,-0.878310,0.878310,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.878309,0.878310,0.878310,NaN,NaN,NaN
scalar_coupling_constant,-0.774582,NaN,-0.878299,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,NaN,NaN,NaN
coupling_distance,0.774624,NaN,0.878331,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,NaN,NaN
coupling_1JHC,-0.774597,NaN,-0.878310,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,NaN,NaN,NaN
coupling_2JHH,0.774597,NaN,0.878310,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,NaN,NaN
atom0_N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom0_F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom0_O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
from models import XGBModel, LGBModel
from models import partition_data

def test(data):
    plt.figure(figsize=(25, 25))
    for i, t in enumerate(sorted(data.type.unique())[0:1], 1):
        data_df = data[data.type == t]
        train_df, test_df = partition_data(data_df)
        
        model = LGBModel(dict(molecules=molecules, 
                              structures=structures),
                         dict(n_jobs=8,
                              max_depth=9,
                              boosting_type='gbdt',
                              num_leaves=128,
                              min_child_samples=20,
                              learning_rate=0.2,
                              n_estimators=1000,
                              reg_alpha=0.1,
                              reg_lambda=0.3,
                              subsample_freq = 1,
                              subsample = 0.9,
                              verbose=500))
  
        #model.corr(train_df, train_df)
        
        print(f'Training on {len(train_df)} elements')
        model.fit(train_df, train_df)
        output, score = model.evaluate(test_df, test_df)
        print(f'{t} score: {score}')
        plt.subplot(3, 3, i)
        plt.plot(test_df.scalar_coupling_constant, output, '*')
        plt.title(t)
    plt.show()
        
test(labelled.head(100))


Training on 17 elements
[[38 'dsgdb9nsd_000007' 5 1 '1JHC' 83.54180145263672 1.0949581861495972
  'H' 'C' 1 0 0 0 1 0 1 0 0 0 0 1 12 0 0 0 0 1 12 0 0 0 0 0 0 0 0 0 0 0 0
  1.0949581861495972 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [50 'dsgdb9nsd_000008' 3 0 '1JHC' 87.62529754638672 2.0782792568206787
  'H' 'C' 1 0 0 0 1 0 1 0 0 0 0 1 12 0 0 0 0 1 12 0 0 0 0 0 0 0 0 0 0 0 0
  1.1003336906433105 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [46 'dsgdb9nsd_000008' 2 0 '1JHC' 87.63259887695312 2.078307867050171
  'H' 'C' 1 0 0 0 1 0 1 0 0 0 0 1 12 0 0 0 0 1 12 0 0 0 0 0 0 0 0 0 0 0 0
  1.100341558456421 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [9 'dsgdb9nsd_000001' 4 0 '1JHC' 84.80950164794922 1.0486942529678345
  'H' 'C' 1 0 0 0 1 0 1 0 0 0 0 1 12 0 0 0 0 1 12 0 0 0 0 0 0 0 0 0 0 0 0
  1.0919475555419922 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [92 'dsgdb9nsd_000011' 5 0 '1JHC' 82.51309967041016 1.1511691808700562
  'H' 'C' 1 0 0 0 1 0 1 0 0 0 0 1 12 0 0 0 0 1 12 0 0 0 0 0 0 0 0 0 0 0 0
  1.0967912673950195 1.0 1.0 0.0 

AttributeError: 'LGBModel' object has no attribute 'output'

<Figure size 1800x1800 with 0 Axes>